In [45]:
import pandas as pd
import numpy as np
from collections import Counter
from copy import copy
from numpy import genfromtxt
from scipy import spatial
from numpy import dot
from numpy.linalg import norm

In [46]:
# read data
categorical_data = genfromtxt('./cleaned_data/categorical_data.csv', delimiter=',')[1:]
bow_data = genfromtxt('./cleaned_data/bow_data.csv', delimiter=',')

data = np.concatenate((categorical_data, bow_data), axis=1)
data.shape

(350, 621)

In [47]:
# cosine similarity
cos_sim = lambda a, b: dot(a, b)/(norm(a)*norm(b))

cos_sim(data[100], data[200])

0.3326495719295578

In [48]:
# jaccard similarity
def jaccard_sim(x, y):
	intersection = np.logical_and(x, y)
	union = np.logical_or(x, y)
	similarity = intersection.sum() / float(union.sum())
	return similarity

jaccard_sim(data[100], data[200])

0.1978021978021978

# algo reference: 

[ similarity funcs ]( https://dataaspirant.com/five-most-popular-similarity-measures-implementation-in-python/ )

[ gale shapely ]( https://towardsdatascience.com/gale-shapley-algorithm-simply-explained-caa344e643c2 )

In [49]:
# duplicate the data

N = len(data)
print(N)
group1_df = pd.DataFrame(data)
group2_df = pd.DataFrame(data)

group1_df.index = [i+1 for i in range(N)]
group2_df.index = [i+1 for i in range(N)]

350


In [50]:
group1_list = [i+1 for i in range(N)]
group2_list = [i+1 for i in range(N)]

In [51]:
women_list = group1_list
man_list = group2_list
women_df = group1_df
man_df = group2_df

In [52]:
# dict to control which women each man can make proposals
women_available = {man:women_list for man in man_list}

# waiting list of men that were able to create pair on each iteration
waiting_list = []

# dict to store created pairs
proposals = {}

# variable to count number of iterations
count = 0

In [53]:
def gale_shapley(sim):
	pass

In [57]:
# while not all men have pairs
while len(waiting_list)<len(man_list):
    # man makes proposals
    for man in man_list:
        if man not in waiting_list:
            # each man make proposal to the top women from it's list
            women = women_available[man]
            try:
                best_choice = man_df.loc[man][man_df.loc[man].index.isin(women)].idxmin()
            except:
                print(man_df.loc[man][man_df.loc[man].index.isin(women)])
                best_choice = 42
            proposals[(man, best_choice)]=(man_df.loc[man][best_choice],
                                                 women_df.loc[man][best_choice])
    # if women have more than one proposals 
    # she will choose the best option
    overlays = Counter([key[1] for key in proposals.keys()])
    # cycle to choose the best options
    for women in overlays.keys():
        if overlays[women]>1:
            # pairs to drop from proposals
            pairs_to_drop = sorted({pair: proposals[pair] for pair in proposals.keys() 
                    if women in pair}.items(), 
                   key=lambda x: x[1][1]
                  )[1:]
            # if man was rejected by woman
            # there is no pint for him to make proposal 
            # second time to the same woman
            for p_to_drop in pairs_to_drop:
                del proposals[p_to_drop[0]]
                _women = copy(women_available[p_to_drop[0][0]])
                _women.remove(p_to_drop[0][1])
                women_available[p_to_drop[0][0]] = _women
    # man who successfully created pairs must be added to the waiting list 
    waiting_list = [man[0] for man in proposals.keys()]
    # update counter
    count+=1

Series([], Name: 202, dtype: float64)
Series([], Name: 239, dtype: float64)


ValueError: list.remove(x): x not in list

In [ ]:
proposals

{('b', 'D'): (2, 2),
 ('d', 'B'): (2, 2),
 ('c', 'A'): (2, 2),
 ('a', 'C'): (3, 2)}

In [109]:
def find(data, sim):
	ret = []
	for i,u1 in enumerate(data):
		u1_friends = { j:sim(u1,u2) for j, u2 in enumerate(data) if i!=j }
		# list friends based on the sim values
		u1_bff = [ (k,v) for k, v in sorted(u1_friends.items(), key=lambda item: item[1], reverse=True) ][:10]
		ret.append(u1_bff)

	return ret

r = find(data, cos_sim)


In [110]:
r[113]

[(167, 0.4465942083663724),
 (315, 0.4393880291048717),
 (296, 0.42256142372325745),
 (52, 0.4101357174954695),
 (192, 0.4057984286223452),
 (146, 0.39831375340784564),
 (261, 0.39779612647907747),
 (245, 0.3929126389928706),
 (171, 0.3924656724215559),
 (279, 0.3791925398112689)]

In [111]:
df = pd.read_csv("Superpowers.csv")

In [112]:
def get_name(i):
	return list(df.iloc[i][["Preferred First Name (Required)", "Last Name / Family Name (Required)"]])

for k, v in r[113]:
	print(get_name(k), v)

['Evan', 'Lin'] 0.4465942083663724
['Zhengxing', 'Xue'] 0.4393880291048717
['Richa', 'Vishwakarma'] 0.42256142372325745
['Jarrett', 'Coleman'] 0.4101357174954695
['Rajesh', 'Mudidana'] 0.4057984286223452
['Joan', 'La Rosa Ferre'] 0.39831375340784564
['Yang', 'Song'] 0.39779612647907747
['Ridhima', 'Sakhuja'] 0.3929126389928706
['Jinxu', 'Liu'] 0.3924656724215559
['Crystal', 'Tang'] 0.3791925398112689


In [101]:
def find_friends(i):
	print_name(i)
	rec = r[i]
	for user, score in rec:
		print_name(user)

find_friends("asdf")

TypeError: Cannot index by location index with a non-integer key